In [ ]:
# !pip3 install -r requirements.txt

In [ ]:
import pandas as pd

## Data Preparation (Loading CSV)

Load the three CSV files into a pandas DataFrame `data`.

In [ ]:
data = pd.read_csv('../final_df.csv')

In [3]:
data.head()

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...


In [4]:
data['sentiment'].value_counts()

sentiment
Positive    7913
Negative    2441
Neutral     1164
Name: count, dtype: int64

In [5]:
data['year'].value_counts()

year
2019    5129
2018    2596
2022    1184
2023    1111
2020     888
2024     514
2021      96
Name: count, dtype: int64

## Convolutional Neural Network

A Convolutional Neural Network (CNN) is a type of deep learning model that is particularly effective for pattern recognition tasks, especially in images and, increasingly, in text. Here’s how a CNN works in principle, broken down into its key components.

Below is an explanation of how a basic CNN works:

1. Convolutional Layer:
	- A CNN’s core layer is the convolutional layer, which applies filters (kernels) to small regions of the input data.
	- For text, a convolutional layer slides filters over sequences of words or tokens. Each filter is designed to detect specific patterns, such as n-grams (e.g., “not good” or “very interesting”) or word sequences relevant to the task.
	- The convolution operation outputs a feature map, where each entry represents the presence or strength of a detected pattern in a specific region of the input.
    
2.	Activation Function (e.g., ReLU):
	- After convolution, an activation function (like ReLU) is applied to introduce non-linearity, allowing the network to model complex patterns.
	- This function essentially “activates” certain features, helping the network focus on meaningful patterns while ignoring less relevant details.

3.	Pooling Layer:
	- A pooling layer (often Global Max Pooling for text data) is applied after convolution to reduce the dimensionality of the feature map, keeping only the most important features.
	- Pooling helps make the network more robust to minor variations and reduces the number of parameters, which speeds up training and helps prevent overfitting.

4.	Fully Connected (Dense) Layer:
	- The pooled features are then passed through one or more fully connected (dense) layers. These layers process the extracted features, combining them to make predictions.
	- In text classification, a final dense layer with a sigmoid or softmax activation is often used to produce a probability score or class label for each input.


### Basic Convolutional Neural Network (Embedding Layer)

- For our task of fake news classification, we add an embedding layer before the convolution layer. An embedding layer is often included to convert words into dense, continuous vector representations (embeddings) that capture semantic relationships.

- An embedding layer provides input that’s suitable for convolution by encoding words as vectors. This way, the CNN can capture patterns in these representations rather than working with raw token IDs.

In [26]:
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Step 1: Tokenization and Padding
max_words = 10000  # Maximum vocabulary size
max_sequence_length = 300  # Maximum length of sequences

# Assuming 'data' is your DataFrame with 'processed_full_review' and 'sentiment' columns
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])

# Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode the sentiment labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y = onehot_encoder.fit_transform(data[['sentiment']])

# Convert y to single-label format for compute_class_weight
y_labels = np.argmax(y, axis=1)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
class_weights_dict = dict(enumerate(class_weights))

# Define the Basic CNN Model with L2 Regularization
def create_basic_cnn():
    model = Sequential()
    
    # Embedding layer
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
    
    # Convolutional layer with L2 regularization
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
    
    # Max pooling layer
    model.add(GlobalMaxPooling1D())
    
    # Fully connected layer with L2 regularization
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))  # Dropout for regularization
    
    # Output layer for three-class classification using softmax
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 2: Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []
f1_scores = []

# Initialize lists to store all true and predicted labels across folds
all_y_true = []
all_y_pred = []

for train_index, val_index in kfold.split(X, y_labels):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = create_basic_cnn()
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, class_weight=class_weights_dict)
    
    # Evaluate the model on the validation set
    y_pred = np.argmax(model.predict(X_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Store predictions and true labels
    all_y_pred.extend(y_pred)
    all_y_true.extend(y_true)
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    accuracy_scores.append(accuracy)
    f1_scores.append(f1)

    print(f"Fold Accuracy: {accuracy:.4f}")
    print(f"Fold F1 Score: {f1:.4f}")

# Print the average cross-validation results
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")

# Compute and print the classification report across all folds
print("\nAverage Classification Report:")
print(classification_report(all_y_true, all_y_pred, target_names=onehot_encoder.categories_[0], digits=4))

Epoch 1/10


/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4988 - loss: 2.0944 - val_accuracy: 0.7878 - val_loss: 0.7660
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7879 - loss: 0.8548 - val_accuracy: 0.7847 - val_loss: 0.6446
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8370 - loss: 0.6746 - val_accuracy: 0.8260 - val_loss: 0.5520
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8734 - loss: 0.5717 - val_accuracy: 0.8112 - val_loss: 0.5895
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9066 - loss: 0.4809 - val_accuracy: 0.8112 - val_loss: 0.6004
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9222 - loss: 0.4204 - val_accuracy: 0.7943 - val_loss: 0.6327
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9353 - loss: 0.3838 - val_accuracy: 0.8095 - val_loss: 0.6568
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9448 - loss: 0.3607 - val_accuracy: 0.8255 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5433 - loss: 2.1004 - val_accuracy: 0.8138 - val_loss: 0.7483
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7623 - loss: 0.8803 - val_accuracy: 0.8047 - val_loss: 0.6556
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8279 - loss: 0.7011 - val_accuracy: 0.8377 - val_loss: 0.5669
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8683 - loss: 0.5895 - val_accuracy: 0.8277 - val_loss: 0.5789
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8928 - loss: 0.5201 - val_accuracy: 0.8034 - val_loss: 0.6295
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9222 - loss: 0.4494 - val_accuracy: 0.8238 - val_loss: 0.5879
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9358 - loss: 0.3986 - val_accuracy: 0.8194 - val_loss: 0.6197
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9520 - loss: 0.3374 - val_accuracy: 0.8424 - val_

...

144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4398 - loss: 2.1185 - val_accuracy: 0.8051 - val_loss: 0.7645
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7901 - loss: 0.8478 - val_accuracy: 0.7834 - val_loss: 0.6542
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8304 - loss: 0.6702 - val_accuracy: 0.7886 - val_loss: 0.6497
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8758 - loss: 0.5703 - val_accuracy: 0.8273 - val_loss: 0.5713
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9005 - loss: 0.5023 - val_accuracy: 0.8056 - val_loss: 0.6343
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9172 - loss: 0.4387 - val_accuracy: 0.8142 - val_loss: 0.6268
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9373 - loss: 0.3799 - val_accuracy: 0.8303 - val_loss: 0.6041
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9527 - loss: 0.3363 - val_accuracy: 0.8286 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5092 - loss: 2.1264 - val_accuracy: 0.7790 - val_loss: 0.7998
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7906 - loss: 0.8417 - val_accuracy: 0.8224 - val_loss: 0.6153
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8417 - loss: 0.6656 - val_accuracy: 0.8359 - val_loss: 0.5705
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8789 - loss: 0.5758 - val_accuracy: 0.8189 - val_loss: 0.6085
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9002 - loss: 0.5008 - val_accuracy: 0.8281 - val_loss: 0.6221
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9205 - loss: 0.4305 - val_accuracy: 0.8254 - val_loss: 0.6644
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9417 - loss: 0.3706 - val_accuracy: 0.8315 - val_loss: 0.6599
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9499 - loss: 0.3362 - val_accuracy: 0.8246 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4122 - loss: 2.1396 - val_accuracy: 0.7455 - val_loss: 0.8344
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7635 - loss: 0.8797 - val_accuracy: 0.7838 - val_loss: 0.6928
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8430 - loss: 0.6750 - val_accuracy: 0.8372 - val_loss: 0.5756
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8797 - loss: 0.5740 - val_accuracy: 0.8489 - val_loss: 0.5260
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9080 - loss: 0.5010 - val_accuracy: 0.8081 - val_loss: 0.6503
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9296 - loss: 0.4153 - val_accuracy: 0.8597 - val_loss: 0.5227
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9513 - loss: 0.3515 - val_accuracy: 0.8485 - val_loss: 0.5657
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9612 - loss: 0.3125 - val_accuracy: 0.8597 - val_

## Convolutional Neural Network + Count Vectorizer

CountVec can be used for CNNs, but its not recommended as countvectorizer loses the meaningful sequential information of the reviews, making our model unable to capture semantic meaning and complex language patterns. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Reshape
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
import random

# Assuming 'data' is your DataFrame with 'processed_full_review' and 'sentiment' columns
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Step 1: Vectorization using CountVectorizer
max_features = 10000  # Maximum vocabulary size
count_vectorizer = CountVectorizer(max_features=max_features)
X_counts = count_vectorizer.fit_transform(data['processed_full_review']).toarray()

# One-hot encode the sentiment labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y = onehot_encoder.fit_transform(data[['sentiment']])

# Convert y to single-label format for compute_class_weight
y_labels = np.argmax(y, axis=1)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
class_weights_dict = dict(enumerate(class_weights))

# Define the CNN Model with L2 Regularization for Count Vectorized Input
def create_cnn_with_countvec(input_shape):
    model = Sequential()
    
    # Reshape input to add a third dimension (needed for Conv1D)
    model.add(Reshape((input_shape, 1), input_shape=(input_shape,)))
    
    # Convolutional layer with L2 regularization
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
    
    # Max pooling layer
    model.add(GlobalMaxPooling1D())
    
    # Fully connected layer with L2 regularization
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))  # Dropout for regularization
    
    # Output layer for three-class classification using softmax
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 2: Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []
f1_scores = []

# Initialize lists to store all true and predicted labels across folds
all_y_true = []
all_y_pred = []

for train_index, val_index in kfold.split(X_counts, y_labels):
    X_train, X_val = X_counts[train_index], X_counts[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = create_cnn_with_countvec(input_shape=X_counts.shape[1])
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, class_weight=class_weights_dict)
    
    # Evaluate the model on the validation set
    y_pred = np.argmax(model.predict(X_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Store predictions and true labels
    all_y_pred.extend(y_pred)
    all_y_true.extend(y_true)
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    accuracy_scores.append(accuracy)
    f1_scores.append(f1)

    print(f"Fold Accuracy: {accuracy:.4f}")
    print(f"Fold F1 Score: {f1:.4f}")

# Print the average cross-validation results
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")

# Compute and print the classification report across all folds
print("\nAverage Classification Report:")
print(classification_report(all_y_true, all_y_pred, target_names=onehot_encoder.categories_[0], digits=4))

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2536 - loss: 1.7359 - val_accuracy: 0.2630 - val_loss: 1.3195
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4360 - loss: 1.2230 - val_accuracy: 0.4583 - val_loss: 1.1956
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4675 - loss: 1.1409 - val_accuracy: 0.5469 - val_loss: 1.1024
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5116 - loss: 1.1078 - val_accuracy: 0.5907 - val_loss: 1.0731
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5767 - loss: 1.0831 - val_accuracy: 0.5885 - val_loss: 1.0788
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5569 - loss: 1.0860 - val_accuracy: 0.5881 - val_loss: 1.0918
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5508 - loss: 1.0874 - val_accuracy: 0.4136 - val_loss: 1.0904
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5493 - loss: 1.0767 - val_accu

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.2848 - loss: 1.7381 - val_accuracy: 0.4349 - val_loss: 1.2859
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4345 - loss: 1.2360 - val_accuracy: 0.4761 - val_loss: 1.1582
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5164 - loss: 1.1268 - val_accuracy: 0.4210 - val_loss: 1.1153
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5499 - loss: 1.0906 - val_accuracy: 0.6159 - val_loss: 1.0764
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5436 - loss: 1.0872 - val_accuracy: 0.4753 - val_loss: 1.0959
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5549 - loss: 1.0881 - val_accuracy: 0.6159 - val_loss: 1.0772
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5633 - loss: 1.0798 - val_accuracy: 0.4753 - val_loss: 1.0912
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5491 - loss: 1.0757 - val_accu

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2802 - loss: 1.7102 - val_accuracy: 0.1415 - val_loss: 1.2992
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3339 - loss: 1.2307 - val_accuracy: 0.5894 - val_loss: 1.1426
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4894 - loss: 1.1384 - val_accuracy: 0.6050 - val_loss: 1.0952
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.4929 - loss: 1.1284 - val_accuracy: 0.6042 - val_loss: 1.0691
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5568 - loss: 1.0941 - val_accuracy: 0.6042 - val_loss: 1.0583
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5451 - loss: 1.0948 - val_accuracy: 0.4089 - val_loss: 1.1079
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5244 - loss: 1.0960 - val_accuracy: 0.4115 - val_loss: 1.0905
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5174 - loss: 1.0978 - val_accu

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2879 - loss: 1.7298 - val_accuracy: 0.4147 - val_loss: 1.2634
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4167 - loss: 1.2247 - val_accuracy: 0.6756 - val_loss: 1.0851
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5232 - loss: 1.1291 - val_accuracy: 0.4815 - val_loss: 1.1333
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5197 - loss: 1.1101 - val_accuracy: 0.6235 - val_loss: 1.0621
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5559 - loss: 1.0808 - val_accuracy: 0.5758 - val_loss: 1.0685
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5338 - loss: 1.0847 - val_accuracy: 0.6231 - val_loss: 1.0661
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5335 - loss: 1.0838 - val_accuracy: 0.4003 - val_loss: 1.0983
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4816 - loss: 1.0899 - val_accu

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.2774 - loss: 1.7265 - val_accuracy: 0.4842 - val_loss: 1.2564
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4721 - loss: 1.2412 - val_accuracy: 0.6135 - val_loss: 1.1254
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5222 - loss: 1.1426 - val_accuracy: 0.6131 - val_loss: 1.0909
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5261 - loss: 1.1199 - val_accuracy: 0.6127 - val_loss: 1.0701
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5663 - loss: 1.0903 - val_accuracy: 0.5636 - val_loss: 1.0614
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5442 - loss: 1.0893 - val_accuracy: 0.5636 - val_loss: 1.0455
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5400 - loss: 1.0719 - val_accuracy: 0.4642 - val_loss: 1.0993
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5242 - loss: 1.0948 - val_accu

## Convolutional Neural Network + Custom-trained Word2Vec embeddings

In this case, we train the embedding layer on our dataset, which allows it to better capture domain-specific vocabulary, as compared to using pre-trained embeddings that are trained on a very large and general corpus.

##### 1. Word embeddings capture the semantic relationships between words in a dense, low-dimensional space.
Fake news often uses subtle language, and word embeddings like GloVe can capture the semantic context of words, allowing the model to understand relationships between words that simple vectorizers would miss. This helps in detecting nuanced differences in language use between real and fake news.

##### 2. Word embeddings produce dense, low-dimensional vectors (e.g., 100-300 dimensions) that capture rich word information.
Pre-trained embeddings are built on large corpora like Wikipedia and news articles, giving our model external knowledge that’s useful for distinguishing between real news and fake news. This boosts the model's ability to generalize on unseen test data from our web scraping.

##### 3. Efficient Representation of Semantics
Words in fake news can appear in different contexts, but with similar underlying meanings (e.g., "hoax" and "lie"). GloVe embeddings represent these similar words in close proximity in the vector space, helping the model recognize fake news patterns more effectively than TF-IDF or Count Vectorizer.

##### 4. Handling Synonyms and Rare Words:
Fake news often uses alternative phrases or rare terminology. Pre-trained embeddings like GloVe can handle these rare words because they’ve seen a broad variety of language during training, making our model more robust against unusual vocabulary choices in fake news.

In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import Word2Vec

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Tokenization parameters
max_words = 10000
max_sequence_length = 300
embedding_dim = 200

# Tokenize and create sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Encode labels and convert to categorical (one-hot encoding)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['sentiment'])
y = to_categorical(y)  # Convert labels to one-hot encoded format

# Step 2: Train custom Word2Vec embeddings
sentences = [text.split() for text in data['processed_full_review']]
custom_word2vec = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=2, workers=4)

# Step 3: Create Embedding Matrix from Custom Word2Vec
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words and word in custom_word2vec.wv:
        embedding_matrix[i] = custom_word2vec.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

# Step 4: Define CNN Model with Custom Word2Vec Embeddings
def create_cnn_with_custom_word2vec():
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size,
                                output_dim=embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)(input_layer)
    
    x = Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01))(embedding_layer)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(3, activation='softmax')(x)  # Output layer for multi-class classification

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 5: Stratified K-Fold Cross-Validation with Class Weights
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []
f1_scores = []
classification_reports = []

# Compute class weights
y_labels = np.argmax(y, axis=1)  # Convert from one-hot to single-label format for class weight computation
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

for train_index, val_index in kfold.split(X, y_labels):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = create_cnn_with_custom_word2vec()
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), 
              class_weight=class_weights_dict, verbose=1)
    
    # Evaluate on validation set
    y_pred = np.argmax(model.predict(X_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # Append scores to the lists
    accuracy_scores.append(accuracy)
    f1_scores.append(f1)

    # Classification report
    report = classification_report(y_true, y_pred, target_names=onehot_encoder.categories_[0], output_dict=True)
    classification_reports.append(report)

Epoch 1/10


/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5984 - loss: 3.0157 - val_accuracy: 0.7556 - val_loss: 1.7562
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7553 - loss: 1.6927 - val_accuracy: 0.7860 - val_loss: 1.1453
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7855 - loss: 1.2064 - val_accuracy: 0.7969 - val_loss: 0.9125
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7957 - loss: 0.9847 - val_accuracy: 0.7756 - val_loss: 0.8460
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8047 - loss: 0.8770 - val_accuracy: 0.8125 - val_loss: 0.7014
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8078 - loss: 0.8094 - val_accuracy: 0.8199 - val_loss: 0.6655
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8169 - loss: 0.7732 - val_accuracy: 0.8390 - val_loss: 0.6177
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8275 - loss: 0.7411 - val_accuracy: 0.8277 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6044 - loss: 2.9858 - val_accuracy: 0.7148 - val_loss: 1.7278
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7393 - loss: 1.5952 - val_accuracy: 0.7487 - val_loss: 1.1425
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7689 - loss: 1.1497 - val_accuracy: 0.7982 - val_loss: 0.8741
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7902 - loss: 0.9606 - val_accuracy: 0.7630 - val_loss: 0.8419
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7999 - loss: 0.8571 - val_accuracy: 0.7604 - val_loss: 0.8097
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7939 - loss: 0.8258 - val_accuracy: 0.7700 - val_loss: 0.7686
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8125 - loss: 0.7655 - val_accuracy: 0.8003 - val_loss: 0.6827
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8118 - loss: 0.7542 - val_accuracy: 0.8103 - val_

...

144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6059 - loss: 2.9026 - val_accuracy: 0.7582 - val_loss: 1.5678
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7642 - loss: 1.4759 - val_accuracy: 0.8025 - val_loss: 0.9643
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7812 - loss: 1.0500 - val_accuracy: 0.7613 - val_loss: 0.8924
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7968 - loss: 0.9053 - val_accuracy: 0.8082 - val_loss: 0.7260
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8087 - loss: 0.8298 - val_accuracy: 0.7999 - val_loss: 0.6911
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8178 - loss: 0.7693 - val_accuracy: 0.8190 - val_loss: 0.6435
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8204 - loss: 0.7465 - val_accuracy: 0.8177 - val_loss: 0.6251
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8250 - loss: 0.7142 - val_accuracy: 0.8181 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6161 - loss: 2.9597 - val_accuracy: 0.7134 - val_loss: 1.6718
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7447 - loss: 1.5338 - val_accuracy: 0.7885 - val_loss: 1.0439
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7790 - loss: 1.0783 - val_accuracy: 0.8094 - val_loss: 0.8372
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7978 - loss: 0.9019 - val_accuracy: 0.8133 - val_loss: 0.7591
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8120 - loss: 0.8301 - val_accuracy: 0.7972 - val_loss: 0.7339
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8003 - loss: 0.7892 - val_accuracy: 0.8254 - val_loss: 0.6577
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8189 - loss: 0.7443 - val_accuracy: 0.8076 - val_loss: 0.6674
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8216 - loss: 0.7304 - val_accuracy: 0.8129 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6352 - loss: 2.9363 - val_accuracy: 0.7703 - val_loss: 1.6386
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7439 - loss: 1.5933 - val_accuracy: 0.7712 - val_loss: 1.1508
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7706 - loss: 1.1504 - val_accuracy: 0.7955 - val_loss: 0.9230
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7823 - loss: 0.9576 - val_accuracy: 0.7586 - val_loss: 0.9156
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.8659 - val_accuracy: 0.7460 - val_loss: 0.9270
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7929 - loss: 0.8191 - val_accuracy: 0.7742 - val_loss: 0.8077
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8075 - loss: 0.7793 - val_accuracy: 0.7620 - val_loss: 0.8354
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8107 - loss: 0.7607 - val_accuracy: 0.7647 - val_

In [29]:
import pandas as pd
import numpy as np

def print_average_classification_report(classification_reports):
    """
    Calculate and print the average metrics across all cross-validation folds.
    
    Parameters:
    classification_reports (list): List of classification_report dictionaries from sklearn
    """
    # Initialize a dictionary to store the sum of metrics
    metrics_sum = {}
    
    # Sum up all metrics across folds
    for report in classification_reports:
        for label, metrics in report.items():
            if isinstance(metrics, dict):  # Skip non-dictionary entries
                if label not in metrics_sum:
                    metrics_sum[label] = {k: 0.0 for k in metrics.keys()}
                for metric_name, value in metrics.items():
                    metrics_sum[label][metric_name] += value
    
    # Calculate averages
    n_folds = len(classification_reports)
    avg_metrics = {
        label: {metric: value/n_folds 
                for metric, value in metrics.items()}
        for label, metrics in metrics_sum.items()
    }
    
    # Create a DataFrame for better formatting
    rows = []
    for label in avg_metrics:
        metrics = avg_metrics[label]
        row = {
            'Label': label,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1-score'],
            'Support': metrics['support']
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Print results
    print("\nAverage Classification Report Across All Folds:")
    print("-" * 80)
    print(df.to_string(index=False, float_format=lambda x: '{:.4f}'.format(x) if isinstance(x, float) else str(x)))
    print("-" * 80)
    
    # Print average accuracy and F1 scores
    print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.4f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.4f}")

# Usage example with your existing code:
print_average_classification_report(classification_reports)


Average Classification Report Across All Folds:
--------------------------------------------------------------------------------
       Label  Precision  Recall  F1-Score   Support
    Negative     0.8054  0.7337    0.7615  488.2000
     Neutral     0.3477  0.6752    0.4541  232.8000
    Positive     0.9612  0.8454    0.8990 1582.6000
   macro avg     0.7048  0.7514    0.7049 2303.6000
weighted avg     0.8662  0.8046    0.8249 2303.6000
--------------------------------------------------------------------------------

Average Accuracy: 0.8046
Average F1-Score: 0.8249


## Convolutional Neural Network + FastText embeddings

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import FastText

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Tokenization parameters
max_words = 10000
max_sequence_length = 300
embedding_dim = 200

# Tokenize and create sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Encode labels and convert to categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['sentiment'])
y = to_categorical(y)

# Train FastText embeddings
sentences = [text.split() for text in data['processed_full_review']]
fasttext_model = FastText(sentences,
                         vector_size=embedding_dim,
                         window=5,
                         min_count=2,
                         workers=4,
                         sg=1)

# Create Embedding Matrix from FastText
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model.wv[word]
        except KeyError:
            embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

# Define CNN Model with FastText Embeddings
def create_cnn_with_fasttext():
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size,
                              output_dim=embedding_dim,
                              weights=[embedding_matrix],
                              input_length=max_sequence_length,
                              trainable=False)(input_layer)
    
    x = Conv1D(filters=128, 
               kernel_size=5, 
               activation='relu', 
               kernel_regularizer=l2(0.01))(embedding_layer)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, 
              activation='relu', 
              kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(3, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists for storing metrics
accuracy_scores = []
f1_scores = []
all_y_true = []
all_y_pred = []

# Compute class weights
y_labels = np.argmax(y, axis=1)
class_weights = compute_class_weight(class_weight='balanced',
                                   classes=np.unique(y_labels),
                                   y=y_labels)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Training and evaluation loop
for fold, (train_index, val_index) in enumerate(kfold.split(X, y_labels)):
    print(f'\nFold {fold + 1}')
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Create and train model
    model = create_cnn_with_fasttext()
    history = model.fit(X_train, y_train,
                       epochs=10,
                       batch_size=64,
                       validation_data=(X_val, y_val),
                       class_weight=class_weights_dict,
                       verbose=1)
    
    # Evaluate on validation set
    y_pred = np.argmax(model.predict(X_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Store predictions and true labels
    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)
    
    # Calculate fold metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    accuracy_scores.append(accuracy)
    f1_scores.append(f1)
    
    print(f'\nFold {fold + 1} Results:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print('\nClassification Report for this fold:')
    print(classification_report(y_true, y_pred, 
                              target_names=label_encoder.classes_,
                              digits=4))

# Print overall results
print('\nOverall Cross-Validation Results:')
print(f'Average Accuracy: {np.mean(accuracy_scores):.4f} (±{np.std(accuracy_scores):.4f})')
print(f'Average F1 Score: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})')

# Print the final classification report on all predictions
print('\nFinal Classification Report on All Folds:')
print(classification_report(all_y_true, 
                          all_y_pred,
                          target_names=label_encoder.classes_,
                          digits=4))


Fold 1
Epoch 1/10


/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5810 - loss: 2.6473 - val_accuracy: 0.8494 - val_loss: 1.0234
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7793 - loss: 1.1274 - val_accuracy: 0.8503 - val_loss: 0.6942
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8122 - loss: 0.8802 - val_accuracy: 0.8529 - val_loss: 0.6161
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8224 - loss: 0.7780 - val_accuracy: 0.8424 - val_loss: 0.5979
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8261 - loss: 0.7332 - val_accuracy: 0.8520 - val_loss: 0.5579
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8315 - loss: 0.7086 - val_accuracy: 0.8550 - val_loss: 0.5481
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8378 - loss: 0.6825 - val_accuracy: 0.8633 - val_loss: 0.5304
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8515 - loss: 0.6622 - val_accuracy: 0.8485 - val

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6037 - loss: 2.5758 - val_accuracy: 0.8242 - val_loss: 1.0057
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7898 - loss: 1.0516 - val_accuracy: 0.8281 - val_loss: 0.7378
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8046 - loss: 0.8443 - val_accuracy: 0.8082 - val_loss: 0.7083
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8238 - loss: 0.7671 - val_accuracy: 0.8225 - val_loss: 0.6423
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8307 - loss: 0.7270 - val_accuracy: 0.8008 - val_loss: 0.6669
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8209 - loss: 0.7128 - val_accuracy: 0.8403 - val_loss: 0.5718
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8329 - loss: 0.6866 - val_accuracy: 0.8459 - val_loss: 0.5533
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8447 - loss: 0.6681 - val_accuracy: 0.8485 - val_

...

144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5878 - loss: 2.5293 - val_accuracy: 0.8147 - val_loss: 0.9824
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7979 - loss: 1.0192 - val_accuracy: 0.8455 - val_loss: 0.6669
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8112 - loss: 0.8224 - val_accuracy: 0.8359 - val_loss: 0.6203
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8258 - loss: 0.7562 - val_accuracy: 0.8529 - val_loss: 0.5680
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8421 - loss: 0.7246 - val_accuracy: 0.8472 - val_loss: 0.5477
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8295 - loss: 0.6968 - val_accuracy: 0.8533 - val_loss: 0.5394
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8441 - loss: 0.6698 - val_accuracy: 0.8472 - val_loss: 0.5426
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8475 - loss: 0.6594 - val_accuracy: 0.8420 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6295 - loss: 2.5422 - val_accuracy: 0.7755 - val_loss: 1.0629
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7975 - loss: 1.0385 - val_accuracy: 0.8133 - val_loss: 0.7734
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8252 - loss: 0.8137 - val_accuracy: 0.8211 - val_loss: 0.6914
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8224 - loss: 0.7452 - val_accuracy: 0.8472 - val_loss: 0.6137
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8435 - loss: 0.7047 - val_accuracy: 0.8267 - val_loss: 0.6342
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8457 - loss: 0.6796 - val_accuracy: 0.7924 - val_loss: 0.6757
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8403 - loss: 0.6756 - val_accuracy: 0.8506 - val_loss: 0.5677
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8550 - loss: 0.6501 - val_accuracy: 0.8354 - val_

/home/dariusng2103/projects/mla_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6421 - loss: 2.5654 - val_accuracy: 0.8419 - val_loss: 1.0142
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7990 - loss: 1.0818 - val_accuracy: 0.8346 - val_loss: 0.7648
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8148 - loss: 0.8634 - val_accuracy: 0.8563 - val_loss: 0.6311
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8219 - loss: 0.7697 - val_accuracy: 0.8050 - val_loss: 0.7251
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8251 - loss: 0.7297 - val_accuracy: 0.8541 - val_loss: 0.5976
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8364 - loss: 0.7050 - val_accuracy: 0.8446 - val_loss: 0.6029
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8402 - loss: 0.6820 - val_accuracy: 0.8159 - val_loss: 0.6660
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8369 - loss: 0.6716 - val_accuracy: 0.8346 - val_